In [1]:
import os
import sys
import copy
import pickle
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import keras.api._v2.keras as keras
import keras.backend as K
from keras.optimizers.legacy import Adam
from keras.optimizers import SGD
from keras import callbacks
from keras.models import Model
from keras.layers import Layer, Dense, Input, Activation, Lambda, BatchNormalization, Conv1D, SpatialDropout1D, add, GlobalAveragePooling1D
from keras.activations import sigmoid
from sklearn.model_selection import KFold
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

In [2]:
# Load multimodal data

from collections import Counter
# emotion_counter = Counter()

def createOneHot(train_label, test_label):
    
    maxlen = int(max(train_label.max(), test_label.max()))

    train = np.zeros((train_label.shape[0], train_label.shape[1], maxlen + 1))
    test = np.zeros((test_label.shape[0], test_label.shape[1], maxlen + 1))

    for i in range(train_label.shape[0]):
        for j in range(train_label.shape[1]):
            train[i, j, train_label[i, j]] = 1

    for i in range(test_label.shape[0]):
        for j in range(test_label.shape[1]):
            test[i, j, test_label[i, j]] = 1

    return train, test

# Load data
f = open("dataset/iemocap/raw/IEMOCAP_features_raw.pkl", "rb")
u = pickle._Unpickler(f)
u.encoding = 'latin1'
videoIDs, videoSpeakers, videoLabels, videoText, videoAudio, videoVisual, videoSentence, trainVid, testVid = u.load()
# label index mapping = {'hap':0, 'sad':1, 'neu':2, 'ang':3, 'exc':4, 'fru':5}

print('Number of training samples: ',len(trainVid))
print('Number of testing samples: ',len(testVid))

train_audio = []
train_text = []
train_visual = []
train_seq_len = []
train_label = []

test_audio = []
test_text = []
test_visual = []
test_seq_len = []
test_label = []
for vid in trainVid:
    train_seq_len.append(len(videoIDs[vid]))
for vid in testVid:
    test_seq_len.append(len(videoIDs[vid]))

max_len = max(max(train_seq_len), max(test_seq_len))
print('Max length of sequences: ', max_len)
print()

for vid in trainVid:
    # emotion_counter.update(videoLabels[vid])
    train_label.append(videoLabels[vid] + [0] * (max_len - len(videoIDs[vid])))
    pad = [np.zeros(videoText[vid][0].shape)] * (max_len - len(videoIDs[vid]))
    text = np.stack(videoText[vid] + pad, axis=0)
    train_text.append(text)

    pad = [np.zeros(videoAudio[vid][0].shape)] * (max_len - len(videoIDs[vid]))
    audio = np.stack(videoAudio[vid] + pad, axis=0)
    train_audio.append(audio)

    pad = [np.zeros(videoVisual[vid][0].shape)] * (max_len - len(videoIDs[vid]))
    video = np.stack(videoVisual[vid] + pad, axis=0)
    train_visual.append(video)

for vid in testVid:
    test_label.append(videoLabels[vid] + [0] * (max_len - len(videoIDs[vid])))
    pad = [np.zeros(videoText[vid][0].shape)] * (max_len - len(videoIDs[vid]))
    text = np.stack(videoText[vid] + pad, axis=0)
    test_text.append(text)

    pad = [np.zeros(videoAudio[vid][0].shape)] * (max_len - len(videoIDs[vid]))
    audio = np.stack(videoAudio[vid] + pad, axis=0)
    test_audio.append(audio)

    pad = [np.zeros(videoVisual[vid][0].shape)] * (max_len - len(videoIDs[vid]))
    video = np.stack(videoVisual[vid] + pad, axis=0)
    test_visual.append(video)

train_text = np.stack(train_text, axis=0)
train_audio = np.stack(train_audio, axis=0)
train_visual = np.stack(train_visual, axis=0)
print(f"Train text shape: {train_text.shape[0]} samples, {train_text.shape[1]} timesteps, {train_text.shape[2]} features")
print(f"Train audio shape: {train_audio.shape[0]} samples, {train_audio.shape[1]} timesteps, {train_audio.shape[2]} features")
print(f"Train visual shape: {train_visual.shape[0]} samples, {train_visual.shape[1]} timesteps, {train_visual.shape[2]} features")
print()

test_text = np.stack(test_text, axis=0)
test_audio = np.stack(test_audio, axis=0)
test_visual = np.stack(test_visual, axis=0)
print(f"Test text shape: {test_text.shape[0]} samples, {test_text.shape[1]} timesteps, {test_text.shape[2]} features")
print(f"Test audio shape: {test_audio.shape[0]} samples, {test_audio.shape[1]} timesteps, {test_audio.shape[2]} features")
print(f"Test visual shape: {test_visual.shape[0]} samples, {test_visual.shape[1]} timesteps, {test_visual.shape[2]} features")
print()

train_label = np.array(train_label)
test_label = np.array(test_label)
train_seq_len = np.array(train_seq_len)
test_seq_len = np.array(test_seq_len)

train_mask = np.zeros((train_text.shape[0], train_text.shape[1]), dtype='float')
for i in range(len(train_seq_len)):
    train_mask[i, :train_seq_len[i]] = 1.0

test_mask = np.zeros((test_text.shape[0], test_text.shape[1]), dtype='float')
for i in range(len(test_seq_len)):
    test_mask[i, :test_seq_len[i]] = 1.0

train_label, test_label = createOneHot(train_label, test_label)
train_data = np.concatenate((train_audio, train_visual, train_text), axis=-1)
test_data = np.concatenate((test_audio, test_visual, test_text), axis=-1)
audio_text_train_data = np.concatenate((train_audio, train_text), axis=-1)
audio_text_test_data = np.concatenate((test_audio, test_text), axis=-1)

print(f"Train data shape: {train_data.shape[0]} samples, {train_data.shape[1]} timesteps, {train_data.shape[2]} features")
print(f"Train label shape: {train_label.shape[0]} samples, {train_label.shape[1]} timesteps, {train_label.shape[2]} classes")
print()
print(f"Test data shape: {test_data.shape[0]} samples, {test_data.shape[1]} timesteps, {test_data.shape[2]} features")
print(f"Test label shape: {test_label.shape[0]} samples, {test_label.shape[1]} timesteps, {test_label.shape[2]} classes")
print()

# for emotion, count in emotion_counter.items():
#   print(f"{emotion}: {count}")

Number of training samples:  120
Number of testing samples:  31
Max length of sequences:  110

Train text shape: 120 samples, 110 timesteps, 100 features
Train audio shape: 120 samples, 110 timesteps, 100 features
Train visual shape: 120 samples, 110 timesteps, 512 features

Test text shape: 31 samples, 110 timesteps, 100 features
Test audio shape: 31 samples, 110 timesteps, 100 features
Test visual shape: 31 samples, 110 timesteps, 512 features

Train data shape: 120 samples, 110 timesteps, 712 features
Train label shape: 120 samples, 110 timesteps, 6 classes

Test data shape: 31 samples, 110 timesteps, 712 features
Test label shape: 31 samples, 110 timesteps, 6 classes



In [4]:
# TIM-Net Model

# Define Common Model class
class Common_Model(object):

    def __init__(self, save_path: str = '', name: str = 'Not Specified'):
        self.model = None
        self.trained = False 

    def train(self, x_train, y_train, x_val, y_val):
        raise NotImplementedError()

    def predict(self, samples):
        raise NotImplementedError()
        

    def predict_proba(self, samples):
        if not self.trained:
            sys.stderr.write("No Model.")
            sys.exit(-1)
        return self.model.predict_proba(samples)

    def save_model(self, model_name: str):
        raise NotImplementedError()


def Temporal_Aware_Block(x, s, i, activation, nb_filters, kernel_size, dropout_rate=0, name=''):

    original_x = x
    #1.1
    conv_1_1 = Conv1D(filters=nb_filters, kernel_size=kernel_size,
                  dilation_rate=i, padding='causal')(x)
    conv_1_1 = BatchNormalization(trainable=True,axis=-1)(conv_1_1)
    conv_1_1 =  Activation(activation)(conv_1_1)
    output_1_1 =  SpatialDropout1D(dropout_rate)(conv_1_1)

    # 2.1
    conv_2_1 = Conv1D(filters=nb_filters, kernel_size=kernel_size,
                  dilation_rate=i, padding='causal')(output_1_1)
    conv_2_1 = BatchNormalization(trainable=True,axis=-1)(conv_2_1)
    conv_2_1 = Activation(activation)(conv_2_1)
    output_2_1 =  SpatialDropout1D(dropout_rate)(conv_2_1)
    
    if original_x.shape[-1] != output_2_1.shape[-1]:
        original_x = Conv1D(filters=nb_filters, kernel_size=1, padding='same')(original_x)
        
    output_2_1 = Lambda(sigmoid)(output_2_1)
    F_x = Lambda(lambda x: tf.multiply(x[0], x[1]))([original_x, output_2_1])
    return F_x


class TIMNET:
    def __init__(self,
                 nb_filters=64,
                 kernel_size=2,
                 nb_stacks=1,
                 dilations=None,
                 activation = "relu",
                 dropout_rate=0.1,
                 return_sequences=True,
                 name='TIMNET'):
        self.name = name
        self.return_sequences = return_sequences
        self.activation = activation
        self.dropout_rate = dropout_rate
        self.dilations = dilations
        self.nb_stacks = nb_stacks
        self.kernel_size = kernel_size
        self.nb_filters = nb_filters

        self.supports_masking = True
        self.mask_value=0.

        if not isinstance(nb_filters, int):
            raise Exception()

    def __call__(self, inputs, mask=None):
        if self.dilations is None:
            self.dilations = 8
        forward = inputs
        backward = K.reverse(inputs,axes=1)
        
        print("Input Shape=",inputs.shape)
        forward_convd = Conv1D(filters=self.nb_filters,kernel_size=1, dilation_rate=1, padding='causal')(forward)
        backward_convd = Conv1D(filters=self.nb_filters,kernel_size=1, dilation_rate=1, padding='causal')(backward)

        final_skip_connection = []
        
        skip_out_forward = forward_convd
        skip_out_backward = backward_convd
        
        for s in range(self.nb_stacks):
            for i in [2 ** i for i in range(self.dilations)]:
                skip_out_forward = Temporal_Aware_Block(skip_out_forward, s, i, self.activation,
                                                        self.nb_filters,
                                                        self.kernel_size, 
                                                        self.dropout_rate,  
                                                        name=self.name)
                skip_out_backward = Temporal_Aware_Block(skip_out_backward, s, i, self.activation,
                                                        self.nb_filters,
                                                        self.kernel_size, 
                                                        self.dropout_rate,  
                                                        name=self.name)
                
                temp_skip = add([skip_out_forward, skip_out_backward],name = "biadd_"+str(i))
                # temp_skip=GlobalAveragePooling1D()(temp_skip)
                # temp_skip=tf.expand_dims(temp_skip, axis=1)
                final_skip_connection.append(temp_skip)

        ''' Commented out for now
        output_2 = final_skip_connection[0]
        for i,item in enumerate(final_skip_connection):
            if i==0:
                continue
            output_2 = K.concatenate([output_2,item],axis=-2)
        x = output_2
        '''

        output_2 = final_skip_connection[0]
        for i, item in enumerate(final_skip_connection[1:]):
            output_2 = K.concatenate([output_2, item], axis=-1)
        x = output_2

        return x

def smooth_labels(labels, factor=0.1):
    # smooth the labels
    labels *= (1 - factor)
    labels += (factor / labels.shape[1])
    return labels

'''
class WeightLayer(Layer):
    def __init__(self, **kwargs):
        super(WeightLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_shape[2],1),
                                      initializer='uniform',
                                      trainable=True)  
        super(WeightLayer, self).build(input_shape)  
 
    def call(self, x):
        tempx = tf.transpose(x,[0,2,1])
        x = K.dot(tempx,self.kernel)
        x = tf.squeeze(x,axis=-1)
        print(f'WeightLayer output shape: {x.shape}')
        return  x
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0],input_shape[1])

'''
class WeightLayer(Layer):
    def build(self, input_shape):
        # This should match the number of features (last dimension of the input shape)
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[-1], 6), 
                                      initializer='uniform', 
                                      trainable=True)
        super(WeightLayer, self).build(input_shape)

    def call(self, x):
        # Perform matrix multiplication without reducing the temporal dimension
        # Assuming self.kernel is [features, output_dim] and x is [batch, timesteps, features]
        x = K.dot(x, self.kernel)
        # Now x should be [batch, timesteps, output_dim]
        return x

    def compute_output_shape(self, input_shape):
        # Preserving the temporal dimension and applying the kernel to the last dimension (features)
        return (input_shape[0], input_shape[1], self.output_dim)

def softmax(x, axis=-1):
    ex = K.exp(x - K.max(x, axis=axis, keepdims=True))
    return ex/K.sum(ex, axis=axis, keepdims=True)

class TIMNET_Model(Common_Model):
    def __init__(self, args, input_shape, class_label, **params):
        super(TIMNET_Model,self).__init__(**params)
        self.args = args
        self.data_shape = input_shape
        self.num_classes = len(class_label)
        self.class_label = class_label
        self.matrix = []
        self.eva_matrix = []
        self.acc = 0
        print("TIMNET MODEL SHAPE:",input_shape)
    
    def create_model(self):
        self.inputs=Input(shape = (self.data_shape[0],self.data_shape[1]))
        self.multi_decision = TIMNET(nb_filters=self.args.filter_size,
                                kernel_size=self.args.kernel_size, 
                                nb_stacks=self.args.stack_size,
                                dilations=self.args.dilation_size,
                                dropout_rate=self.args.dropout,
                                activation = self.args.activation,
                                return_sequences=True, 
                                name='TIMNET')(self.inputs)
        print(f'Multi_decision output shape: {self.multi_decision.shape}')

        self.decision = WeightLayer()(self.multi_decision)
        self.predictions = Dense(self.num_classes, activation='softmax')(self.decision)
        self.model = Model(inputs = self.inputs, outputs = self.predictions)
        
        self.model.compile(loss = "categorical_crossentropy",
                           optimizer =Adam(learning_rate=self.args.lr, beta_1=self.args.beta1, beta_2=self.args.beta2, epsilon=1e-8),
                           metrics = ['accuracy'])
        print("TIMNet Model creation completed")
        
    def train(self, x, y):
        label_mapping = {'hap': 0, 'sad': 1, 'neu': 2, 'ang': 3, 'exc': 4, 'fru': 5}
        reverse_mapping = {v: k for k, v in label_mapping.items()}

        filepath = self.args.model_path
        resultpath = self.args.result_path

        os.makedirs(filepath, exist_ok=True)
        os.makedirs(resultpath, exist_ok=True)

        now = datetime.datetime.now()
        now_time = datetime.datetime.strftime(now,'%Y-%m-%d_%H-%M-%S')

        total_accuracy = 0
        total_loss = 0
        total_precision = []
        total_recall = []
        total_f1 = []

        kfold = KFold(n_splits=self.args.split_fold, shuffle=True, random_state=self.args.random_seed)
        
        fold_counter = 0
        
        for train_index, test_index in kfold.split(x):
            self.create_model()
            y_train_smoothed = smooth_labels(y[train_index].copy(), 0.1)
            
            folder_address = os.path.join(filepath, f"{self.args.data}_{self.args.random_seed}_{now_time}")
            os.makedirs(folder_address, exist_ok=True)
            
            weight_path = os.path.join(folder_address, f"{self.args.split_fold}-fold_weights_best_{fold_counter+1}.hdf5")
            checkpoint = ModelCheckpoint(weight_path, verbose=1, save_weights_only=True, save_best_only=True)

            self.model.fit(x[train_index], y_train_smoothed, validation_data=(x[test_index], y[test_index]),
                            batch_size=self.args.batch_size, epochs=self.args.epoch, verbose=1, callbacks=[checkpoint])
            
            self.model.load_weights(weight_path)
            eval_result = self.model.evaluate(x[test_index], y[test_index], verbose=0)
            
            total_loss += eval_result[0]
            total_accuracy += eval_result[1]

            # Generate predictions for the current fold's test set
            y_pred = self.model.predict(x[test_index])
            y_true = y[test_index]
            y_pred_classes = np.argmax(y_pred, axis=-1).flatten()  # Convert softmax outputs to class indices and flatten
            y_true_classes = np.argmax(y[test_index], axis=-1).flatten()  # Correctly using y[test_index] to flatten the true labels

            # Generate the classification report for the current fold
            try:
                print(classification_report(y_true_classes, y_pred_classes, target_names=[reverse_mapping[i] for i in range(len(label_mapping))]))
            except Exception as e:
                print(f"Error generating classification report: {e}")

            try:
                precision, recall, f1, _ = precision_recall_fscore_support(y_true_classes, y_pred_classes, average='weighted')
                total_precision.append(precision)
                total_recall.append(recall)
                total_f1.append(f1)
            except Exception as e:
                print(f"Error generating precision, recall, and f1-score: {e}")

            fold_counter += 1

        avg_accuracy = total_accuracy / self.args.split_fold
        avg_loss = total_loss / self.args.split_fold
        print(f"Average accuracy over {self.args.split_fold} folds: {avg_accuracy}")
        print(f"Average loss over {self.args.split_fold} folds: {avg_loss}")
        avg_precision = np.mean(total_precision)
        avg_recall = np.mean(total_recall)
        avg_f1 = np.mean(total_f1)

        print(f"Average Precision: {avg_precision}")
        print(f"Average Recall: {avg_recall}")
        print(f"Average F1-Score: {avg_f1}")

        # Cleanup
        K.clear_session()

    def test(self, x, y, path):
        label_mapping = {'hap': 0, 'sad': 1, 'neu': 2, 'ang': 3, 'exc': 4, 'fru': 5}
        reverse_mapping = {v: k for k, v in label_mapping.items()}

        kfold = KFold(n_splits=self.args.split_fold, shuffle=True, random_state=self.args.random_seed)

        total_accuracy = []
        total_precision = []
        total_recall = []
        total_f1 = []
        fold_counter = 0

        for train_index, test_index in kfold.split(x):
            fold_counter += 1
            self.create_model()
            weight_path = os.path.join(path, f"{self.args.split_fold}-fold_weights_best_{fold_counter}.hdf5")
            self.model.load_weights(weight_path)

            eval_result = self.model.evaluate(x[test_index], y[test_index], verbose=0)
            total_accuracy.append(eval_result[1])

            y_pred = self.model.predict(x[test_index])
            y_pred_classes = np.argmax(y_pred, axis=-1).flatten()
            y_true_classes = np.argmax(y[test_index], axis=-1).flatten()

            try:
                precision, recall, f1, _ = precision_recall_fscore_support(y_true_classes, y_pred_classes, average='weighted')
                total_precision.append(precision)
                total_recall.append(recall)
                total_f1.append(f1)

                print(f"Fold {fold_counter} Classification Report:\n")
                print(classification_report(y_true_classes, y_pred_classes, target_names=[reverse_mapping[i] for i in range(len(label_mapping))]))

            except Exception as e:
                print(f"Error generating metrics for fold {fold_counter}: {e}")

        # Calculating average metrics
        avg_accuracy = np.mean(total_accuracy)
        avg_precision = np.mean(total_precision)
        avg_recall = np.mean(total_recall)
        avg_f1 = np.mean(total_f1)

        print(f"Average Accuracy: {avg_accuracy}")
        print(f"Average Precision: {avg_precision}")
        print(f"Average Recall: {avg_recall}")
        print(f"Average F1-Score: {avg_f1}")

        # Cleanup
        K.clear_session()


In [5]:
# Define and set parameters
class Args:
    def __init__(self, mode, data='IEMOCAP', split_fold=5, random_seed=16, epoch=50, test_path='./Test_Models/IEMOCAP_16'):
        self.mode = mode
        self.data = data
        self.split_fold = split_fold
        self.random_seed = random_seed
        self.epoch = epoch
        self.gpu = 0
        self.model_path = './Models/'
        self.result_path = './Results/'
        self.test_path = test_path
        self.lr = 0.001
        self.beta1 = 0.93
        self.beta2 = 0.98
        self.batch_size = 64
        self.dropout = 0.1
        self.activation = 'relu'
        self.filter_size = 39
        self.dilation_size = 8 
        self.kernel_size = 2
        self.stack_size = 1


In [11]:
# Train with only text data
class_label = ["happy", "sad", "neutral", "angry", "excited", "frustrated"]
args_train = Args(mode="train")
model = TIMNET_Model(args=args_train, input_shape=train_text.shape[1:], class_label=class_label)
model.train(train_text, train_label)

TIMNET MODEL SHAPE: (110, 100)
Input Shape= (None, 110, 100)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed
Epoch 1/50


2024-03-12 21:00:54.390176: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - ETA: 0s - loss: 1.9477 - accuracy: 0.3512 

2024-03-12 21:01:54.254947: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_loss improved from inf to 1.73644, saving model to ./Models/IEMOCAP_16_2024-03-12_21-00-49/5-fold_weights_best_1.hdf5
2/2 [==============================] - 74s 35s/step - loss: 1.9477 - accuracy: 0.3512 - val_loss: 1.7364 - val_accuracy: 0.5102
Epoch 2/50
2/2 [==============================] - ETA: 0s - loss: 1.4197 - accuracy: 0.5216
Epoch 2: val_loss improved from 1.73644 to 1.26068, saving model to ./Models/IEMOCAP_16_2024-03-12_21-00-49/5-fold_weights_best_1.hdf5
2/2 [==============================] - 17s 9s/step - loss: 1.4197 - accuracy: 0.5216 - val_loss: 1.2607 - val_accuracy: 0.6739
Epoch 3/50
2/2 [==============================] - ETA: 0s - loss: 1.1031 - accuracy: 0.6730
Epoch 3: val_loss improved from 1.26068 to 1.00203, saving model to ./Models/IEMOCAP_16_2024-03-12_21-00-49/5-fold_weights_best_1.hdf5
2/2 [==============================] - 16s 8s/step - loss: 1.1031 - accuracy: 0.6730 - val_loss: 1.0020 - val_accuracy: 0.7818
Epoch 4/50
2/2 [================

...

2/2 [==============================] - ETA: 0s - loss: 2.0404 - accuracy: 0.3592  

2024-03-12 21:21:54.206624: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_loss improved from inf to 1.81166, saving model to ./Models/IEMOCAP_16_2024-03-12_21-00-49/5-fold_weights_best_5.hdf5
2/2 [==============================] - 117s 56s/step - loss: 2.0404 - accuracy: 0.3592 - val_loss: 1.8117 - val_accuracy: 0.5314
Epoch 2/50
2/2 [==============================] - ETA: 0s - loss: 1.4915 - accuracy: 0.4987
Epoch 2: val_loss improved from 1.81166 to 1.38063, saving model to ./Models/IEMOCAP_16_2024-03-12_21-00-49/5-fold_weights_best_5.hdf5
2/2 [==============================] - 20s 11s/step - loss: 1.4915 - accuracy: 0.4987 - val_loss: 1.3806 - val_accuracy: 0.6833
Epoch 3/50
2/2 [==============================] - ETA: 0s - loss: 1.1707 - accuracy: 0.6546
Epoch 3: val_loss improved from 1.38063 to 1.15849, saving model to ./Models/IEMOCAP_16_2024-03-12_21-00-49/5-fold_weights_best_5.hdf5
2/2 [==============================] - 21s 12s/step - loss: 1.1707 - accuracy: 0.6546 - val_loss: 1.1585 - val_accuracy: 0.7803
Epoch 4/50
2/2 [=============

2024-03-12 21:26:00.242586: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 15s 15s/step
              precision    recall  f1-score   support

         hap       0.93      0.99      0.96      1570
         sad       0.89      0.91      0.90       111
         neu       0.84      0.79      0.82       182
         ang       0.88      0.83      0.86       255
         exc       0.86      0.66      0.75       199
         fru       0.93      0.80      0.86       323

    accuracy                           0.91      2640
   macro avg       0.89      0.83      0.86      2640
weighted avg       0.91      0.91      0.91      2640

Average accuracy over 5 folds: 0.9334848523139954
Average loss over 5 folds: 0.597497183084488
Average Precision: 0.9346317630720756
Average Recall: 0.9334848484848484
Average F1-Score: 0.932858631523216


In [12]:
# Test with unseen text-only data
args_test = Args(mode="test", test_path='./Models/IEMOCAP_16_2024-03-12_21-00-49')
model = TIMNET_Model(args=args_test, input_shape=test_text.shape[1:], class_label=["happy", "sad", "neutral", "angry", "excited", "frustrated"])
model.test(test_text, test_label, path=args_test.test_path)

TIMNET MODEL SHAPE: (110, 100)
Input Shape= (None, 110, 100)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


2024-03-12 21:30:34.848622: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-12 21:30:48.899001: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 13s 13s/step
Fold 1 Classification Report:

              precision    recall  f1-score   support

         hap       0.99      0.93      0.96       505
         sad       0.58      0.45      0.51        40
         neu       0.30      0.41      0.35        39
         ang       0.53      0.64      0.58        47
         exc       0.71      0.58      0.64        64
         fru       0.46      0.61      0.53        75

    accuracy                           0.80       770
   macro avg       0.60      0.60      0.59       770
weighted avg       0.83      0.80      0.81       770

Input Shape= (None, 110, 100)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


2024-03-12 21:31:02.990593: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-12 21:31:16.790548: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 12s 12s/step
Fold 2 Classification Report:

              precision    recall  f1-score   support

         hap       0.94      0.99      0.96       371
         sad       0.67      0.65      0.66        96
         neu       0.44      0.26      0.33        72
         ang       0.16      0.42      0.23        12
         exc       0.41      0.33      0.37        36
         fru       0.53      0.53      0.53        73

    accuracy                           0.76       660
   macro avg       0.52      0.53      0.51       660
weighted avg       0.76      0.76      0.76       660

Input Shape= (None, 110, 100)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


...

1/1 [==============================] - 13s 13s/step
Fold 3 Classification Report:

              precision    recall  f1-score   support

         hap       0.79      0.94      0.86       301
         sad       0.20      0.29      0.24         7
         neu       0.45      0.61      0.52       115
         ang       0.32      0.36      0.34        25
         exc       0.74      0.40      0.52       151
         fru       0.47      0.23      0.31        61

    accuracy                           0.66       660
   macro avg       0.50      0.47      0.46       660
weighted avg       0.67      0.66      0.65       660

Input Shape= (None, 110, 100)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


2024-03-12 21:32:01.629900: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-12 21:32:15.019531: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 12s 12s/step
Fold 4 Classification Report:

              precision    recall  f1-score   support

         hap       0.83      0.96      0.89       381
         sad       0.62      0.50      0.55        48
         neu       0.50      0.28      0.36        93
         ang       0.21      0.67      0.32        15
         exc       0.38      0.24      0.29        46
         fru       0.56      0.40      0.47        77

    accuracy                           0.71       660
   macro avg       0.52      0.51      0.48       660
weighted avg       0.69      0.71      0.69       660

Input Shape= (None, 110, 100)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


2024-03-12 21:32:28.097518: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-12 21:32:41.540644: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 17s 17s/step
Fold 5 Classification Report:

              precision    recall  f1-score   support

         hap       0.98      0.99      0.99       373
         sad       0.87      0.48      0.62        54
         neu       0.47      0.48      0.47        65
         ang       0.62      0.55      0.58        71
         exc       0.06      0.50      0.10         2
         fru       0.55      0.62      0.58        95

    accuracy                           0.80       660
   macro avg       0.59      0.60      0.56       660
weighted avg       0.82      0.80      0.80       660

Average Accuracy: 0.7468398332595825
Average Precision: 0.7534824228448553
Average Recall: 0.7468398268398267
Average F1-Score: 0.7410389722310592


In [6]:
# Train with only audio data
class_label = ["happy", "sad", "neutral", "angry", "excited", "frustrated"]
args_train = Args(mode="train")
model = TIMNET_Model(args=args_train, input_shape=train_audio.shape[1:], class_label=class_label)
model.train(train_audio, train_label)

TIMNET MODEL SHAPE: (110, 100)
Input Shape= (None, 110, 100)


2024-03-12 20:07:53.252257: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-03-12 20:07:53.252275: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-12 20:07:53.252282: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-12 20:07:53.252318: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-12 20:07:53.252334: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed
Epoch 1/50


2024-03-12 20:07:56.582377: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - ETA: 0s - loss: 1.6148 - accuracy: 0.4021

...

2024-03-12 20:17:32.815867: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_loss improved from inf to 1.77570, saving model to ./Models/IEMOCAP_16_2024-03-12_20-07-53/5-fold_weights_best_5.hdf5
2/2 [==============================] - 35s 14s/step - loss: 1.6154 - accuracy: 0.4092 - val_loss: 1.7757 - val_accuracy: 0.6144
Epoch 2/50
2/2 [==============================] - ETA: 0s - loss: 1.5931 - accuracy: 0.6296
Epoch 2: val_loss improved from 1.77570 to 1.75933, saving model to ./Models/IEMOCAP_16_2024-03-12_20-07-53/5-fold_weights_best_5.hdf5
2/2 [==============================] - 6s 4s/step - loss: 1.5931 - accuracy: 0.6296 - val_loss: 1.7593 - val_accuracy: 0.6223
Epoch 3/50
2/2 [==============================] - ETA: 0s - loss: 1.5715 - accuracy: 0.6351
Epoch 3: val_loss improved from 1.75933 to 1.74161, saving model to ./Models/IEMOCAP_16_2024-03-12_20-07-53/5-fold_weights_best_5.hdf5
2/2 [==============================] - 5s 3s/step - loss: 1.5715 - accuracy: 0.6351 - val_loss: 1.7416 - val_accuracy: 0.6284
Epoch 4/50
2/2 [==================

2024-03-12 20:18:58.064161: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 5s 5s/step
              precision    recall  f1-score   support

         hap       0.75      0.99      0.85      1570
         sad       0.55      0.66      0.60       111
         neu       0.31      0.05      0.09       182
         ang       0.47      0.58      0.52       255
         exc       0.00      0.00      0.00       199
         fru       0.24      0.06      0.09       323

    accuracy                           0.68      2640
   macro avg       0.39      0.39      0.36      2640
weighted avg       0.56      0.68      0.60      2640

Average accuracy over 5 folds: 0.6837121248245239
Average loss over 5 folds: 0.9522679448127747
Average Precision: 0.6312167520564888
Average Recall: 0.6837121212121213
Average F1-Score: 0.6240116180530311


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [8]:
# Test with unseen audio-only data
args_test = Args(mode="test", test_path='./Models/IEMOCAP_16_2024-03-12_20-07-53')
model = TIMNET_Model(args=args_test, input_shape=test_audio.shape[1:], class_label=["happy", "sad", "neutral", "angry", "excited", "frustrated"])
model.test(test_audio, test_label, path=args_test.test_path)

TIMNET MODEL SHAPE: (110, 100)
Input Shape= (None, 110, 100)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


2024-03-12 20:24:03.099049: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-12 20:24:07.887678: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 4s 4s/step
Fold 1 Classification Report:

              precision    recall  f1-score   support

         hap       0.84      0.95      0.89       505
         sad       0.38      0.45      0.41        40
         neu       0.00      0.00      0.00        39
         ang       1.00      0.09      0.16        47
         exc       0.50      0.06      0.11        64
         fru       0.29      0.53      0.37        75

    accuracy                           0.71       770
   macro avg       0.50      0.35      0.32       770
weighted avg       0.70      0.71      0.66       770

Input Shape= (None, 110, 100)


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


...

1/1 [==============================] - 7s 7s/step
Fold 4 Classification Report:

              precision    recall  f1-score   support

         hap       0.74      0.95      0.83       381
         sad       0.33      0.02      0.04        48
         neu       0.28      0.22      0.24        93
         ang       0.17      0.20      0.18        15
         exc       0.52      0.50      0.51        46
         fru       0.53      0.23      0.32        77

    accuracy                           0.65       660
   macro avg       0.43      0.35      0.35       660
weighted avg       0.59      0.65      0.59       660

Input Shape= (None, 110, 100)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


2024-03-12 20:24:46.185374: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2024-03-12 20:24:51.068248: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 5s 5s/step
Fold 5 Classification Report:

              precision    recall  f1-score   support

         hap       0.76      0.99      0.86       373
         sad       0.67      0.56      0.61        54
         neu       0.25      0.03      0.05        65
         ang       0.41      0.65      0.50        71
         exc       0.00      0.00      0.00         2
         fru       0.50      0.04      0.08        95

    accuracy                           0.68       660
   macro avg       0.43      0.38      0.35       660
weighted avg       0.62      0.68      0.60       660

Average Accuracy: 0.650649344921112
Average Precision: 0.6073471480239376
Average Recall: 0.6506493506493506
Average F1-Score: 0.5884835715496669


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [9]:
# Train with video data only
class_label = ["happy", "sad", "neutral", "angry", "excited", "frustrated"]
args_train = Args(mode="train")
model = TIMNET_Model(args=args_train, input_shape=train_visual.shape[1:], class_label=class_label)
model.train(train_visual, train_label)

TIMNET MODEL SHAPE: (110, 512)
Input Shape= (None, 110, 512)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed
Epoch 1/50


2024-03-12 20:38:00.051118: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - ETA: 0s - loss: 1.6195 - accuracy: 0.4709 

2024-03-12 20:38:32.862233: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_loss improved from inf to 1.78227, saving model to ./Models/IEMOCAP_16_2024-03-12_20-37-53/5-fold_weights_best_1.hdf5
2/2 [==============================] - 43s 17s/step - loss: 1.6195 - accuracy: 0.4709 - val_loss: 1.7823 - val_accuracy: 0.6610
Epoch 2/50
2/2 [==============================] - ETA: 0s - loss: 1.6097 - accuracy: 0.6312
Epoch 2: val_loss improved from 1.78227 to 1.77311, saving model to ./Models/IEMOCAP_16_2024-03-12_20-37-53/5-fold_weights_best_1.hdf5
2/2 [==============================] - 7s 4s/step - loss: 1.6097 - accuracy: 0.6312 - val_loss: 1.7731 - val_accuracy: 0.6693
Epoch 3/50
2/2 [==============================] - ETA: 0s - loss: 1.5988 - accuracy: 0.6363
Epoch 3: val_loss improved from 1.77311 to 1.76274, saving model to ./Models/IEMOCAP_16_2024-03-12_20-37-53/5-fold_weights_best_1.hdf5
2/2 [==============================] - 6s 3s/step - loss: 1.5988 - accuracy: 0.6363 - val_loss: 1.7627 - val_accuracy: 0.6705
Epoch 4/50
2/2 [==================

...

2024-03-12 20:48:27.996227: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_loss improved from inf to 1.78867, saving model to ./Models/IEMOCAP_16_2024-03-12_20-37-53/5-fold_weights_best_5.hdf5
2/2 [==============================] - 60s 31s/step - loss: 1.6228 - accuracy: 0.2586 - val_loss: 1.7887 - val_accuracy: 0.6625
Epoch 2/50
2/2 [==============================] - ETA: 0s - loss: 1.6181 - accuracy: 0.6409
Epoch 2: val_loss improved from 1.78867 to 1.78474, saving model to ./Models/IEMOCAP_16_2024-03-12_20-37-53/5-fold_weights_best_5.hdf5
2/2 [==============================] - 14s 7s/step - loss: 1.6181 - accuracy: 0.6409 - val_loss: 1.7847 - val_accuracy: 0.6170
Epoch 3/50
2/2 [==============================] - ETA: 0s - loss: 1.6132 - accuracy: 0.6581
Epoch 3: val_loss improved from 1.78474 to 1.78037, saving model to ./Models/IEMOCAP_16_2024-03-12_20-37-53/5-fold_weights_best_5.hdf5
2/2 [==============================] - 11s 7s/step - loss: 1.6132 - accuracy: 0.6581 - val_loss: 1.7804 - val_accuracy: 0.6011
Epoch 4/50
2/2 [================

2024-03-12 20:50:48.622395: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 8s 8s/step
              precision    recall  f1-score   support

         hap       0.74      0.99      0.85      1570
         sad       0.00      0.00      0.00       111
         neu       0.00      0.00      0.00       182
         ang       0.00      0.00      0.00       255
         exc       0.00      0.00      0.00       199
         fru       0.42      0.67      0.51       323

    accuracy                           0.67      2640
   macro avg       0.19      0.28      0.23      2640
weighted avg       0.49      0.67      0.57      2640

Average accuracy over 5 folds: 0.6573484897613525
Average loss over 5 folds: 0.9596872806549073
Average Precision: 0.5487609156908886
Average Recall: 0.657348484848485
Average F1-Score: 0.5878016860316515


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [10]:
# Test with video data only
args_test = Args(mode="test", test_path='./Models/IEMOCAP_16_2024-03-12_20-37-53')
model = TIMNET_Model(args=args_test, input_shape=test_visual.shape[1:], class_label=["happy", "sad", "neutral", "angry", "excited", "frustrated"])
model.test(test_visual, test_label, path=args_test.test_path)

TIMNET MODEL SHAPE: (110, 512)
Input Shape= (None, 110, 512)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


2024-03-12 20:53:32.280510: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-12 20:53:41.805920: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 8s 8s/step
Fold 1 Classification Report:

              precision    recall  f1-score   support

         hap       0.82      0.97      0.89       505
         sad       0.00      0.00      0.00        40
         neu       0.00      0.00      0.00        39
         ang       0.00      0.00      0.00        47
         exc       0.00      0.00      0.00        64
         fru       0.29      0.67      0.41        75

    accuracy                           0.70       770
   macro avg       0.19      0.27      0.22       770
weighted avg       0.57      0.70      0.62       770

Input Shape= (None, 110, 512)


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


...

/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


2024-03-12 20:54:50.481743: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-12 20:54:59.800555: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 10s 10s/step
Fold 5 Classification Report:

              precision    recall  f1-score   support

         hap       0.76      0.99      0.86       373
         sad       0.00      0.00      0.00        54
         neu       0.00      0.00      0.00        65
         ang       0.00      0.00      0.00        71
         exc       0.00      0.00      0.00         2
         fru       0.36      0.60      0.45        95

    accuracy                           0.65       660
   macro avg       0.19      0.27      0.22       660
weighted avg       0.48      0.65      0.55       660

Average Accuracy: 0.6376190543174743
Average Precision: 0.49897484752318483
Average Recall: 0.6376190476190476
Average F1-Score: 0.5547373832448232


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [15]:
# Train with audio and text data
class_label = ["happy", "sad", "neutral", "angry", "excited", "frustrated"]
args_train = Args(mode="train")
model = TIMNET_Model(args=args_train, input_shape=audio_text_train_data.shape[1:], class_label=class_label)
model.train(audio_text_train_data, train_label)

TIMNET MODEL SHAPE: (110, 200)
Input Shape= (None, 110, 200)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed
Epoch 1/50


2024-03-12 22:41:29.464328: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - ETA: 0s - loss: 1.8896 - accuracy: 0.3654  

2024-03-12 22:43:52.551339: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_loss improved from inf to 1.55202, saving model to ./Models/IEMOCAP_16_2024-03-12_22-41-23/5-fold_weights_best_1.hdf5
2/2 [==============================] - 170s 81s/step - loss: 1.8896 - accuracy: 0.3654 - val_loss: 1.5520 - val_accuracy: 0.5508
Epoch 2/50
2/2 [==============================] - ETA: 0s - loss: 1.4448 - accuracy: 0.5002 
Epoch 2: val_loss improved from 1.55202 to 1.22953, saving model to ./Models/IEMOCAP_16_2024-03-12_22-41-23/5-fold_weights_best_1.hdf5
2/2 [==============================] - 40s 21s/step - loss: 1.4448 - accuracy: 0.5002 - val_loss: 1.2295 - val_accuracy: 0.6716
Epoch 3/50
2/2 [==============================] - ETA: 0s - loss: 1.1848 - accuracy: 0.6231 
Epoch 3: val_loss improved from 1.22953 to 1.03519, saving model to ./Models/IEMOCAP_16_2024-03-12_22-41-23/5-fold_weights_best_1.hdf5
2/2 [==============================] - 34s 19s/step - loss: 1.1848 - accuracy: 0.6231 - val_loss: 1.0352 - val_accuracy: 0.7655
Epoch 4/50
2/2 [===========

...

2/2 [==============================] - ETA: 0s - loss: 2.0246 - accuracy: 0.3139  

2024-03-12 23:30:00.834210: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_loss improved from inf to 1.81323, saving model to ./Models/IEMOCAP_16_2024-03-12_22-41-23/5-fold_weights_best_5.hdf5
2/2 [==============================] - 533s 441s/step - loss: 2.0246 - accuracy: 0.3139 - val_loss: 1.8132 - val_accuracy: 0.5678
Epoch 2/50
2/2 [==============================] - ETA: 0s - loss: 1.4866 - accuracy: 0.5090 
Epoch 2: val_loss improved from 1.81323 to 1.35788, saving model to ./Models/IEMOCAP_16_2024-03-12_22-41-23/5-fold_weights_best_5.hdf5
2/2 [==============================] - 51s 26s/step - loss: 1.4866 - accuracy: 0.5090 - val_loss: 1.3579 - val_accuracy: 0.7117
Epoch 3/50
2/2 [==============================] - ETA: 0s - loss: 1.1645 - accuracy: 0.6702 
Epoch 3: val_loss improved from 1.35788 to 1.12181, saving model to ./Models/IEMOCAP_16_2024-03-12_22-41-23/5-fold_weights_best_5.hdf5
2/2 [==============================] - 47s 24s/step - loss: 1.1645 - accuracy: 0.6702 - val_loss: 1.1218 - val_accuracy: 0.8000
Epoch 4/50
2/2 [==========

2024-03-12 23:37:16.100907: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 23s 23s/step
              precision    recall  f1-score   support

         hap       0.97      1.00      0.98      1570
         sad       0.88      0.93      0.90       111
         neu       0.82      0.87      0.84       182
         ang       0.93      0.88      0.90       255
         exc       0.96      0.78      0.86       199
         fru       0.91      0.89      0.90       323

    accuracy                           0.94      2640
   macro avg       0.91      0.89      0.90      2640
weighted avg       0.94      0.94      0.94      2640

Average accuracy over 5 folds: 0.9452272653579712
Average loss over 5 folds: 0.5503374993801117
Average Precision: 0.9454900042820599
Average Recall: 0.9452272727272728
Average F1-Score: 0.9449971943945142


In [17]:
# Test audio and text with unseen data
args_test = Args(mode="test", test_path='./Models/IEMOCAP_16_2024-03-12_22-41-23')
model = TIMNET_Model(args=args_test, input_shape=audio_text_test_data.shape[1:], class_label=["happy", "sad", "neutral", "angry", "excited", "frustrated"])
model.test(audio_text_test_data, test_label, path=args_test.test_path)

TIMNET MODEL SHAPE: (110, 200)
Input Shape= (None, 110, 200)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


2024-03-12 23:38:08.520310: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-12 23:38:35.482668: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 25s 25s/step
Fold 1 Classification Report:

              precision    recall  f1-score   support

         hap       0.98      0.94      0.96       505
         sad       0.59      0.47      0.53        40
         neu       0.25      0.38      0.30        39
         ang       0.55      0.72      0.62        47
         exc       0.63      0.56      0.60        64
         fru       0.55      0.57      0.56        75

    accuracy                           0.81       770
   macro avg       0.59      0.61      0.60       770
weighted avg       0.83      0.81      0.81       770

Input Shape= (None, 110, 200)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


2024-03-12 23:39:02.318323: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-12 23:39:28.864984: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 26s 26s/step
Fold 2 Classification Report:

              precision    recall  f1-score   support

         hap       0.92      0.99      0.96       371
         sad       0.82      0.52      0.64        96
         neu       0.43      0.40      0.41        72
         ang       0.06      0.08      0.07        12
         exc       0.52      0.31      0.39        36
         fru       0.51      0.66      0.57        73

    accuracy                           0.77       660
   macro avg       0.54      0.49      0.51       660
weighted avg       0.77      0.77      0.76       660

Input Shape= (None, 110, 200)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


...

1/1 [==============================] - 23s 23s/step
Fold 3 Classification Report:

              precision    recall  f1-score   support

         hap       0.82      0.91      0.86       301
         sad       0.05      0.43      0.09         7
         neu       0.52      0.38      0.44       115
         ang       0.44      0.28      0.34        25
         exc       0.65      0.36      0.46       151
         fru       0.40      0.52      0.45        61

    accuracy                           0.63       660
   macro avg       0.48      0.48      0.44       660
weighted avg       0.67      0.63      0.63       660

Input Shape= (None, 110, 200)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


2024-03-12 23:40:46.123254: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-12 23:41:09.021637: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 23s 23s/step
Fold 4 Classification Report:

              precision    recall  f1-score   support

         hap       0.90      0.91      0.91       381
         sad       0.39      0.60      0.47        48
         neu       0.51      0.26      0.34        93
         ang       0.27      0.60      0.37        15
         exc       0.49      0.37      0.42        46
         fru       0.44      0.49      0.47        77

    accuracy                           0.70       660
   macro avg       0.50      0.54      0.50       660
weighted avg       0.71      0.70      0.70       660

Input Shape= (None, 110, 200)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


2024-03-12 23:41:37.724197: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-12 23:42:04.413316: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 24s 24s/step
Fold 5 Classification Report:

              precision    recall  f1-score   support

         hap       0.99      1.00      1.00       373
         sad       0.82      0.50      0.62        54
         neu       0.55      0.58      0.57        65
         ang       0.69      0.56      0.62        71
         exc       0.14      0.50      0.22         2
         fru       0.56      0.71      0.63        95

    accuracy                           0.83       660
   macro avg       0.63      0.64      0.61       660
weighted avg       0.84      0.83      0.83       660

Average Accuracy: 0.7455844044685364
Average Precision: 0.7636734791832757
Average Recall: 0.7455844155844156
Average F1-Score: 0.7463475262569499


In [13]:
# Train with all data
class_label = ["happy", "sad", "neutral", "angry", "excited", "frustrated"]
args_train = Args(mode="train")
model = TIMNET_Model(args=args_train, input_shape=train_data.shape[1:], class_label=class_label)
model.train(train_data, train_label)

TIMNET MODEL SHAPE: (110, 712)
Input Shape= (None, 110, 712)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed
Epoch 1/50


2024-03-12 21:51:14.508736: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - ETA: 0s - loss: 1.6801 - accuracy: 0.3866 

2024-03-12 21:52:40.187498: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_loss improved from inf to 1.46179, saving model to ./Models/IEMOCAP_16_2024-03-12_21-51-09/5-fold_weights_best_1.hdf5
2/2 [==============================] - 102s 48s/step - loss: 1.6801 - accuracy: 0.3866 - val_loss: 1.4618 - val_accuracy: 0.6508
Epoch 2/50
2/2 [==============================] - ETA: 0s - loss: 1.3321 - accuracy: 0.6153 
Epoch 2: val_loss improved from 1.46179 to 1.16681, saving model to ./Models/IEMOCAP_16_2024-03-12_21-51-09/5-fold_weights_best_1.hdf5
2/2 [==============================] - 24s 12s/step - loss: 1.3321 - accuracy: 0.6153 - val_loss: 1.1668 - val_accuracy: 0.8318
Epoch 3/50
2/2 [==============================] - ETA: 0s - loss: 1.0849 - accuracy: 0.7908 
Epoch 3: val_loss improved from 1.16681 to 0.96316, saving model to ./Models/IEMOCAP_16_2024-03-12_21-51-09/5-fold_weights_best_1.hdf5
2/2 [==============================] - 25s 13s/step - loss: 1.0849 - accuracy: 0.7908 - val_loss: 0.9632 - val_accuracy: 0.8947
Epoch 4/50
2/2 [===========

...

2/2 [==============================] - ETA: 0s - loss: 1.6401 - accuracy: 0.3556  

2024-03-12 22:26:01.058738: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_loss improved from inf to 1.48310, saving model to ./Models/IEMOCAP_16_2024-03-12_21-51-09/5-fold_weights_best_5.hdf5
2/2 [==============================] - 208s 99s/step - loss: 1.6401 - accuracy: 0.3556 - val_loss: 1.4831 - val_accuracy: 0.6004
Epoch 2/50
2/2 [==============================] - ETA: 0s - loss: 1.2604 - accuracy: 0.6244 
Epoch 2: val_loss improved from 1.48310 to 1.23484, saving model to ./Models/IEMOCAP_16_2024-03-12_21-51-09/5-fold_weights_best_5.hdf5
2/2 [==============================] - 50s 27s/step - loss: 1.2604 - accuracy: 0.6244 - val_loss: 1.2348 - val_accuracy: 0.7886
Epoch 3/50
2/2 [==============================] - ETA: 0s - loss: 1.0042 - accuracy: 0.8119 
Epoch 3: val_loss improved from 1.23484 to 1.09177, saving model to ./Models/IEMOCAP_16_2024-03-12_21-51-09/5-fold_weights_best_5.hdf5
2/2 [==============================] - 48s 22s/step - loss: 1.0042 - accuracy: 0.8119 - val_loss: 1.0918 - val_accuracy: 0.8390
Epoch 4/50
2/2 [===========

2024-03-12 22:33:58.056695: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 24s 24s/step
              precision    recall  f1-score   support

         hap       0.98      1.00      0.99      1570
         sad       0.82      0.95      0.88       111
         neu       0.73      0.88      0.80       182
         ang       0.93      0.88      0.90       255
         exc       0.97      0.71      0.82       199
         fru       0.92      0.85      0.89       323

    accuracy                           0.94      2640
   macro avg       0.89      0.88      0.88      2640
weighted avg       0.94      0.94      0.94      2640

Average accuracy over 5 folds: 0.9426515102386475
Average loss over 5 folds: 0.5516646265983581
Average Precision: 0.9439111233756815
Average Recall: 0.9426515151515151
Average F1-Score: 0.9421918849772393


In [14]:
# Test audio, video and text with unseen data
args_test = Args(mode="test", test_path='Models/IEMOCAP_16_2024-03-12_21-51-09')
model = TIMNET_Model(args=args_test, input_shape=test_data.shape[1:], class_label=["happy", "sad", "neutral", "angry", "excited", "frustrated"])
model.test(test_data, test_label, path=args_test.test_path)

TIMNET MODEL SHAPE: (110, 712)
Input Shape= (None, 110, 712)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


2024-03-12 22:34:30.433143: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-12 22:34:55.557130: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 28s 28s/step
Fold 1 Classification Report:

              precision    recall  f1-score   support

         hap       0.98      0.94      0.96       505
         sad       0.79      0.55      0.65        40
         neu       0.28      0.44      0.34        39
         ang       0.52      0.77      0.62        47
         exc       0.65      0.58      0.61        64
         fru       0.64      0.63      0.64        75

    accuracy                           0.82       770
   macro avg       0.64      0.65      0.64       770
weighted avg       0.85      0.82      0.83       770

Input Shape= (None, 110, 712)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


2024-03-12 22:35:26.327653: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-12 22:35:53.177828: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 25s 25s/step
Fold 2 Classification Report:

              precision    recall  f1-score   support

         hap       0.88      0.99      0.93       371
         sad       0.73      0.62      0.67        96
         neu       0.34      0.19      0.25        72
         ang       0.00      0.00      0.00        12
         exc       0.38      0.33      0.35        36
         fru       0.54      0.59      0.57        73

    accuracy                           0.75       660
   macro avg       0.48      0.46      0.46       660
weighted avg       0.72      0.75      0.73       660

Input Shape= (None, 110, 712)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


...

1/1 [==============================] - 24s 24s/step
Fold 3 Classification Report:

              precision    recall  f1-score   support

         hap       0.79      0.94      0.86       301
         sad       0.12      0.29      0.17         7
         neu       0.45      0.47      0.46       115
         ang       0.37      0.28      0.32        25
         exc       0.77      0.37      0.50       151
         fru       0.34      0.41      0.37        61

    accuracy                           0.65       660
   macro avg       0.47      0.46      0.45       660
weighted avg       0.66      0.65      0.63       660

Input Shape= (None, 110, 712)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


2024-03-12 22:37:07.514077: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-12 22:37:34.066592: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 22s 22s/step
Fold 4 Classification Report:

              precision    recall  f1-score   support

         hap       0.91      0.94      0.92       381
         sad       0.62      0.58      0.60        48
         neu       0.55      0.39      0.45        93
         ang       0.43      0.67      0.53        15
         exc       0.44      0.15      0.23        46
         fru       0.41      0.62      0.50        77

    accuracy                           0.74       660
   macro avg       0.56      0.56      0.54       660
weighted avg       0.73      0.74      0.73       660

Input Shape= (None, 110, 712)
Multi_decision output shape: (None, 110, 312)
TIMNet Model creation completed


2024-03-12 22:38:01.703533: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-12 22:38:26.705206: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 23s 23s/step
Fold 5 Classification Report:

              precision    recall  f1-score   support

         hap       0.99      0.99      0.99       373
         sad       0.84      0.50      0.63        54
         neu       0.49      0.57      0.53        65
         ang       0.71      0.56      0.63        71
         exc       0.25      0.50      0.33         2
         fru       0.58      0.72      0.64        95

    accuracy                           0.82       660
   macro avg       0.64      0.64      0.63       660
weighted avg       0.84      0.82      0.82       660

Average Accuracy: 0.7565367937088012
Average Precision: 0.7592443949556318
Average Recall: 0.7565367965367965
Average F1-Score: 0.7489922367312374
